In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

# Load prepared dataframes from pickle files
data_path = "../data_processed/"

with open(data_path + "df_prepared.pkl", "rb") as f:
    df = pickle.load(f)

with open(data_path + "merged_df_prepared.pkl", "rb") as f:
    merged_df = pickle.load(f)

print("DataFrames loaded successfully:")
print(f"  df shape: {df.shape}")
print(f"  merged_df shape: {merged_df.shape}")

# Descriptives

# a) KPIs

## KPI 1: Utilization Rate

The utlization rate rate show us which portion of the charging station is being used at a time. This KPI helps us to get a better understanding of demand & user patterns. When the utilization is high then normally the demand is also high which may suggest to expand capacity at a site or provide good support for the given time frame. In correspondace to that does a low utlization rate means low demand and therefore highlights inefficiencies at a given time or site.

$$
\text{Utilization Rate} = \frac{\text{time charging stations were in usesage}}{\text{total time available for usesage}}
$$

In [ ]:
def calculate_utilizationrate (df, date) :
    possible_session_times = df[df['connectionTime'].dt.date == pd.to_datetime(date).date()]
    possible_session_times = possible_session_times.copy()
    possible_session_times['session_duration'] = (possible_session_times['disconnectTime'] - possible_session_times['connectionTime']) / pd.Timedelta(hours=1)

    total_stations = len(set(df['stationID']))
    total_available_time = total_stations * 24
    utilized_time_station = possible_session_times.groupby('stationID')['session_duration'].sum()
    utilized_time_station = utilized_time_station.apply(lambda x: min(x, 24))
    total_utilized_time = utilized_time_station.sum()
    utilization_rate = (total_utilized_time / total_available_time) * 100
    return utilization_rate


def calculate_utilizationrate_per_day (df):
    dates = df['connectionTime'].dt.date.unique()
    utilization_rates = []
    for date in dates:
        utilization_rate = calculate_utilizationrate(df, date)
        utilization_rates.append({'date': date, 'utilization_rate': utilization_rate})

    utilization_rates = sorted(utilization_rates, key=lambda x: x['date'])
    return pd.DataFrame(utilization_rates)

daily_utilization_df = calculate_utilizationrate_per_day(df)
daily_utilization_df.set_index('date', inplace=True)
print(daily_utilization_df)



In [ ]:
util_df = calculate_utilizationrate_per_day(df)
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(
    util_df['date'],
    util_df['utilization_rate'],
    marker='o',
    linewidth=1.5,
    color='tab:blue'
)

ax.set_xlabel('Date')
ax.set_ylabel('Utilization Rate (%)')
ax.set_title('Daily Utilization Rate of Charging Stations')

plt.show()

The plot show clear changes in charging station utilization over time, with distinct phases of higher and lower daliy usage, as well as visible gaps where utilization drops close to zero. These pattern suggest that both temporal trens and external events influence how intensively the stations are used.
In the earlier time frame until the start of 2019 the utlization gradually increases to a moderate level, indicating a growing adoption of the charging stations. After this is the utilization is at a more consistent niveau, where many days have a utlization rate of above 25-30%, until early 2020. Then the utilization have a huge dropoff to around zero, indicating a disruption or strongly reduced demand over several month and then slowly starting to raise again until the end of our time frame, which a short between mid 2020 to early 2021 where no data is recorded, indicating a complete shutdown.

The large day-to-day fluctuations around each trend level suggest that utilization is not uniform accross the week, with somme days frequently showing much higher usage than others. This make it reasonble to hypthesize that weekdays and weekends differ in charging behavior. For example, commuters mightcharge moreon weekdays, while leisure or shopping trips could shift some demand to weekdays.
To better understand these differences and how they interact with the daily trend, the analysis first zooms in to the utilization profile within a day (utilization per hour) and then compares weekday versus weekend usage on that finer time scale.


### Utilization per hour

In [ ]:
def calculate_hourly_utilization(df):
    
    # Create a copy with the extracted hour
    df_hours = df.copy()
    
    hourly_usage = []
    
    for idx, row in df_hours.iterrows():
        start = row['connectionTime']
        end = row['doneChargingTime']
        station = row['stationID']
        
        # Generate all hours this session occupied
        current = start.replace(minute=0, second=0, microsecond=0)
        while current < end:
            hourly_usage.append({
                'datetime': current,
                'hour': current.hour,
                'date': current.date(),
                'stationID': station
            })
            current += pd.Timedelta(hours=1)
    
    usage_df = pd.DataFrame(hourly_usage)
    
    # Count unique station-hour combinations per hour of day
    total_stations = df['stationID'].nunique()
    total_days = (df['connectionTime'].max().date() - df['connectionTime'].min().date()).days + 1
    
    # Group by hour and count unique station-date combinations
    hourly_stats = usage_df.groupby('hour').agg({
        'stationID': 'count'  # Total station-hours occupied
    }).reset_index()
    
    # Total available station-hours per hour = total_stations * total_days
    hourly_stats['total_available'] = total_stations * total_days
    hourly_stats['utilization_rate'] = (hourly_stats['stationID'] / hourly_stats['total_available']) * 100
    
    return hourly_stats

In [ ]:
hourly_util = calculate_hourly_utilization(df)
print(hourly_util)
# Visualization
fig, ax = plt.subplots(figsize=(12, 6))

ax.bar(hourly_util['hour'], hourly_util['utilization_rate'], 
       color='tab:blue', alpha=0.7, edgecolor='black')

ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Utilization Rate (%)', fontsize=12)
ax.set_title('Average Utilization Rate by Hour of Day', fontsize=14, fontweight='bold')
ax.set_xticks(range(0, 24))
ax.set_xticklabels([f'{h:02d}:00' for h in range(24)], rotation=45, ha='right')
ax.grid(axis='y', alpha=0.3)

# Highlight peak hours
peak_hour = hourly_util.loc[hourly_util['utilization_rate'].idxmax(), 'hour']
ax.axvline(peak_hour, color='red', linestyle='--', alpha=0.5, label=f'Peak hour: {peak_hour:02d}:00')
ax.legend()

plt.tight_layout()
plt.show()

The hourly utilization shows a very pronounced morning peak that dominates the entire daily pattern. During the night hours from 00:00 to 05:00, utilization is almost negligible, indicating that overnight demand at these stations is very small. Starting at around 06:00, utilization rises sharply until it reaches the maximum at 09:00, highlighted by the red dashed line in the chart.
After the peak, utilization decreases at a graduate leveluntil midnight. the missing of a second peak in the late afternoon or evening indicates that charging demand is much more strongly linked to the start of the day than to an evening return-home pattern.

From an operational perspective, the interval from 08:00 to 11:00 can be seen as the critcal window where demand regularly pushes closest to available capacity. if users experience queues or congestion, intervenitions such as adding more chargers, introducng time-limited parking or pricing incentives to shift some charging to other hours would be most efficient when targeted at this peak period.


### Weekydays vs Weekends

To test this hypothesis, a comparison between weekday and weekend utilization should be performed by calculating the respective metrics like mean and standard deviation. If weekdays show consistently higher average utilization than weekends, this would provide quantitative evidence for the effect of day type on charging station usage, beyond the overall time trnds visible in the plot

In [ ]:
# Use column is_weekend for differentiation into weekdays and weekends
util_df['day_of_week'] = pd.to_datetime(util_df['date']).dt.day_name()
util_df['day_name_short'] = pd.to_datetime(util_df['date']).dt.strftime('%a')
util_df['is_weekend'] = util_df['day_of_week'].isin(['Saturday', 'Sunday'])

# Calculate statistics for each day of the week
print("=" * 60)
print("UTILIZATION RATE BY DAY OF WEEK")
print("=" * 60)
by_day = util_df.groupby('day_of_week')['utilization_rate'].agg(['mean', 'std', 'count'])
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
by_day = by_day.reindex(day_order)
print(by_day)

print("\n" + "=" * 60)
print("WEEKDAY vs WEEKEND")
print("=" * 60)
by_weekend = util_df.groupby('is_weekend')['utilization_rate'].agg(['mean', 'std', 'count'])
by_weekend.index = ['Weekday', 'Weekend']
print(by_weekend)

# Calculate difference between weekday and weekend averages
weekday_mean = util_df[~util_df['is_weekend']]['utilization_rate'].mean()
weekend_mean = util_df[util_df['is_weekend']]['utilization_rate'].mean()
diff_pct = ((weekday_mean - weekend_mean) / weekend_mean) * 100
print(f"\nWeekday average: {weekday_mean:.2f}%")
print(f"Weekend average: {weekend_mean:.2f}%")
print(f"Difference: {diff_pct:.1f}% higher on weekdays")


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Box plot for each day of week
util_df_sorted = util_df.copy()
util_df_sorted['day_of_week'] = pd.Categorical(util_df_sorted['day_of_week'], categories=day_order, ordered=True)
util_df_sorted.boxplot(column='utilization_rate', by='day_of_week', ax=ax1)
ax1.set_xlabel('Day of Week')
ax1.set_ylabel('Utilization Rate (%)')
ax1.set_title('Utilization Rate by Day of Week')
plt.sca(ax1)
plt.xticks(rotation=45)

# Box plot for Weekday vs Weekend
ax2.boxplot([util_df[~util_df['is_weekend']]['utilization_rate'], 
              util_df[util_df['is_weekend']]['utilization_rate']],
            tick_labels=['Weekday', 'Weekend'])
ax2.set_ylabel('Utilization Rate (%)')
ax2.set_title('Utilization Rate: Weekday vs Weekend')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

When comparing weekdays and weekends, the average utilization is dramatically higher on weekdays, confirming our hypothesis that most charging activity happens during the week. The summary statistics show a weekday mean of 18% utilization compared to roughly 2.8% on weekends, which corresponds to utilization being severely higher on weekdays, not just marginally higher.

The breakdown by the day of the week indicates that all weekdays lay around similar mean utilization levels of roughly 14-20%, while the weekend days are both quite low with means of around 3%. The aggregated view on weekday and weekend supports this further.



### Rolling averages and confidence bands
To get a better understanding at how these patterns evolve over time we use a rolling average with confidence bands. Using a rolling average smooths out daily noise and in addition by adding confidence bands around the rolling average, it becomes possible to distinguish short-term random fluctuations from meaning gul structural changes in utilization

In [ ]:
util_df["date"] = pd.to_datetime(util_df["date"])

window = 30  # rolling window of 30 days

# Rolling statistics for mean and standard deviation
roll_mean = util_df["utilization_rate"].rolling(window, min_periods=10).mean()
roll_std  = util_df["utilization_rate"].rolling(window, min_periods=10).std()

# mean ± 2 std (≈95%) as confidence band
k = 2
upper = roll_mean + k * roll_std
lower = roll_mean - k * roll_std

fig, ax = plt.subplots(figsize=(10, 4))

# Original daily values (light)
ax.plot(util_df["date"], util_df["utilization_rate"],
        color="tab:blue", alpha=0.3, label="Daily utilization")

# Rolling mean
ax.plot(util_df["date"], roll_mean,
        color="tab:blue", linewidth=2, label=f"{window}-day rolling mean")

# Confidence band
ax.fill_between(util_df["date"], lower, upper,
                color="tab:blue", alpha=0.15, label=f"±{k}·rolling std")

ax.set_xlabel("Date")
ax.set_ylabel("Utilization rate (%)")
ax.set_title("Daily utilization with rolling average and band")
ax.grid(True, linestyle="--", alpha=0.4)
ax.legend()
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

The light band around the rolling mean represents ±2 times the rolling standard deviation and therefore captures typical variability in utilization around the underlying trend. A wide band indicates strong fluctuations in utilization, suggesting unstable or rapidly changing usage conditions. Narrow bands indicate more homogeneous and predictable use. The visualization reveals distinct phases: a gradual increase in 2018, a pronounced peak toward late 2018, a stable high‑utilization plateau through much of 2019, a sharp drop around early 2020, and then a slow recovery followed by renewed growth into 2021.

The rolling patterns naturally invite a seasonal interpretation, as many of the rises and fall recur around the same calendar periods. For instance, the increase from mid-year toward late-year followed by a leveling decline may correspond to seasonal factors such as weather, holiday travel, or annual rythmus in coomuting behavior. This implies that certain seasons systematically bring higher charging demand than other

### Seasonal utilization rate



In [ ]:
# Define seasons based on meteorological seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # 9, 10, 11
        return 'Fall'

# Add column for season
util_df['month'] = pd.to_datetime(util_df['date']).dt.month
util_df['season'] = util_df['month'].apply(get_season)

# Calculate statistics for each season
print("=" * 60)
print("UTILIZATION RATE BY SEASON")
print("=" * 60)
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
by_season = util_df.groupby('season')['utilization_rate'].agg(['mean', 'median', 'std', 'count'])
by_season = by_season.reindex(season_order)
print(by_season)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Box plot for each season
util_df_season = util_df.copy()
util_df_season['season'] = pd.Categorical(util_df_season['season'], categories=season_order, ordered=True)
util_df_season.boxplot(column='utilization_rate', by='season', ax=ax1)
ax1.set_xlabel('Season')
ax1.set_ylabel('Utilization Rate (%)')
ax1.set_title('Utilization Rate by Season')
ax1.get_figure().suptitle('')  # Remove default title

# Bar chart with mean values
season_means = util_df.groupby('season')['utilization_rate'].mean().reindex(season_order)
colors = ['#87CEEB', '#90EE90', '#FFD700', '#FF8C00']  # Winter, Spring, Summer, Fall
ax2.bar(season_order, season_means, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Season')
ax2.set_ylabel('Average Utilization Rate (%)')
ax2.set_title('Average Utilization Rate by Season')
ax2.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (season, value) in enumerate(zip(season_order, season_means)):
    ax2.text(i, value + 0.5, f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Time series colored by season
fig, ax = plt.subplots(figsize=(14, 6))

for season, color in zip(season_order, colors):
    mask = util_df['season'] == season
    ax.scatter(util_df[mask]['date'], util_df[mask]['utilization_rate'], 
               label=season, color=color, alpha=0.6, s=20)

ax.set_xlabel('Date')
ax.set_ylabel('Utilization Rate (%)')
ax.set_title('Daily Utilization Rate colored by Season')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.4)
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

# Statistical test: Check if differences are significant
print("\n" + "=" * 60)
print("SEASON COMPARISON")
print("=" * 60)
winter_mean = util_df[util_df['season'] == 'Winter']['utilization_rate'].mean()
spring_mean = util_df[util_df['season'] == 'Spring']['utilization_rate'].mean()
summer_mean = util_df[util_df['season'] == 'Summer']['utilization_rate'].mean()
fall_mean = util_df[util_df['season'] == 'Fall']['utilization_rate'].mean()

print(f"Winter: {winter_mean:.2f}%")
print(f"Spring: {spring_mean:.2f}%")
print(f"Summer: {summer_mean:.2f}%")
print(f"Fall:   {fall_mean:.2f}%")

highest_season = by_season['mean'].idxmax()
lowest_season = by_season['mean'].idxmin()
diff = by_season.loc[highest_season, 'mean'] - by_season.loc[lowest_season, 'mean']
print(f"\nHighest: {highest_season} ({by_season.loc[highest_season, 'mean']:.2f}%)")
print(f"Lowest:  {lowest_season} ({by_season.loc[lowest_season, 'mean']:.2f}%)")
print(f"Difference: {diff:.2f} percentage points")

The seasonal statistics show that the  winter, spring and summer cluster around the same average utilization levels, with means of roughly 11-14%. This suggests a broadly comparable demand across these seasons. 
In contrast, fall stands out clearly with an average utilization of around 20%. This is substantially higher than the other seasons and signal a distinct peak in charging activity during fall.

The boxplot of the average utilization by season reinforces this picture. While the interquartile ranges for winter, spring, and summer overlap strongly and are centered around mid-single to low-double digit utilization, the fall box is shifted upward, with both its median and upper quartle at noticably higher levels. The implication of this is that not only occasional peak days but also typical fall days experiencehigher utilization, making fall structually busier rathern than being deiven by few outliers.

The scatterplot show how the seasonal trends play out over time. The data points of fall frequently reach higher utilization values, particularly in the pre-2020 phase and form dense clusters at eleveated levels compared to the other seasons. Even during later years, when ovveral utilization is lower, fall days still tend to populate the upper part of the seasonal cloud. This is consistent with the summary statistics and confirm that fall is the period of systematically strongest usage of charging stations.

## KPI 2: Energy delivered per Hour

In [ ]:
def hourlyEnergyDelivered(df, group_cols=None):
    # Create a copy with hour extracted
    if group_cols is None:
        group_cols = []
    
    df_hours = df.copy()
    
    hourly_energy = []
    
    for idx, row in df_hours.iterrows():
        start = row['connectionTime']
        end = row['disconnectTime']
        energy = row['kWhDelivered']  # in kWh
        site = row['siteID']
        
        # Calculate total session duration in hours
        total_duration_hours = (end - start).total_seconds() / 3600.0
        
        # Already handled in cleaning phase
        if total_duration_hours == 0:
            continue  # Skip sessions with zero duration
        
        # Energy delivered per hour (uniform rate across session hours)
        energy_per_hour = energy / total_duration_hours
        
        # Generate all hours this session occupied
        current = start.replace(minute=0, second=0, microsecond=0)
        while current < end:
            rec = {
                'datetime': current,
                'hour': current.hour,
                'date': current.date(),
                'year': current.year,
                'siteID': site,
                'energy_delivered': energy_per_hour
            }
            for col in group_cols:
                rec[col] = row[col]
            hourly_energy.append(rec)
            current += pd.Timedelta(hours=1)
    
    energy_df = pd.DataFrame(hourly_energy)

    by_cols = ['year', 'hour'] + list(group_cols)
    
    # Aggregate energy delivered per exact hourly datetime
    hourly_stats = energy_df.groupby(by_cols, as_index=False)['energy_delivered'].sum({
    })
    
    return hourly_stats

In [ ]:
print(hourlyEnergyDelivered(df))


### Differentiation between Years

In [ ]:
hourly_energy = hourlyEnergyDelivered(df)
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#87CEEB', '#90EE90', '#FFD700', '#FF8C00']  # Colors for different years#
line_styles = ['-', '--', '-.', ':']

for idx, year in enumerate(sorted(hourly_energy['year'].unique())):
    year_data = hourly_energy[hourly_energy['year'] == year]
    ax.plot(
        year_data['hour'],
        year_data['energy_delivered'],
        marker='o',
        label=str(year),
        color=colors[idx % len(colors)],
        linestyle=line_styles[idx % len(line_styles)],
        linewidth=2
    )
ax.set_title('Hourly Energy Delivered by Year')
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Total Energy Delivered (kWh)')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.4)

The chart confirms that the same patterns seen in the first KPI are present in the hourly energy delivered, supporting a strong correlation between the two.
2019 shows the highest deliverd energy with more than double those the other years during peak hours. 2018 has a more moderate profile being lower than 2019 but above 2020 and 2021. 2020 has consistently the lowest curve, confirmingit as the year with the least delivered energy. 2021 lies between 2018 and 2020, the delivered energy is higher than 2020 during most of the day but does not reach the level of 2018.

All years show a similar shape over the day with very low energy delivered during the night, a sharp increase during the morning, a plateau from midday to the afternoon and a decline towards the evening.

### Differentiation between Sites

In [ ]:
hourly_energy_site = hourlyEnergyDelivered(df, group_cols=['siteID'])
hourly_energy_site_all_years = hourly_energy_site.groupby(['hour', 'siteID'], as_index=False)['energy_delivered'].sum()

fig, ax = plt.subplots(figsize=(14, 7))
colors = ['#87CEEB', '#90EE90']  # Colors for different years#
line_styles = ['-', ':']

for idx, site in enumerate(sorted(hourly_energy_site_all_years['siteID'].unique())):
    site_data = hourly_energy_site_all_years[hourly_energy_site_all_years['siteID'] == site]
    ax.plot(
        site_data['hour'],
        site_data['energy_delivered'],
        marker='o',
        label=f'Site {site}',
        color=colors[idx % len(colors)],
        linestyle=line_styles[idx % len(line_styles)],
        linewidth=2
    )

ax.set_title('Hourly Energy Delivered per Site')
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Total Energy Delivered (kWh)')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.4)

Looking at the differentiation between the sites, we also see the same patterns of the day as before. The energy delivered by site 1 is during the peek hour quite higher than for site 2, except in the evening and early night where the energy delivered for site 2 is higher. We will look in detail in part b of this task.

# b) Site Characteristics